In [28]:
from Bio import SeqIO
from Bio.Seq import Seq

In [48]:
class GapTracker:
    def __init__(self):
        self.names = ["query", "sbjct"]
        self.gap = [None for _ in range(len(self.names))]
        self.gaps = [[] for _ in range(len(self.names))]
        
    def start(self, i, name):
        idx = self.names.index(name)
        if self.gap[idx] is None:
            self.gap[idx] = {"start": i}
            
    def end(self, i, name):
        idx = self.names.index(name)
        if self.gap[idx] is not None:
            self.gap[idx]["end"] = i
            self.gaps[idx].append(self.gap[idx])
            self.gap[idx] = None

In [72]:
alignFilepath = "/home/mitsuki/altorf/denovo/trg-trace/align/mycobacterium/g062726/g063019-cds4882.align"
rec_lst = [rec for rec in SeqIO.parse(alignFilepath, "fasta")]
assert len(rec_lst) == 2
query = rec_lst[0].seq
sbjct = rec_lst[1].seq
assert len(query) == len(sbjct)
length = len(query)

In [73]:
qi = 0
codon = ""
gt = GapTracker()
codon_lst = []

for i in range(length):
    if query[i] == "-":
        codon = None
        gt.start(i, "query")
        gt.end(i, "sbjct")
    elif sbjct[i] == "-":
        qi += 1
        codon = None
        gt.start(i, "sbjct")
        gt.end(i, "query")
    else:
        qi += 1
        if codon is not None:
            codon += sbjct[i]
        gt.end(i, "query")
        gt.end(i, "sbjct")

    if qi > 0 and qi%3 == 0:
        if codon is not None:
            codon_lst.append(codon)
        codon = ""
gt.end(length, "query")
gt.end(length, "sbjct")

In [75]:
gt.gaps

[[{'end': 327, 'start': 326}], []]

In [70]:
codon_lst[0]

'gcg'

In [74]:
for codon in codon_lst:
    print(Seq(codon).translate(), end="")

MPEALRELSGAWNFRDVADGAPMLRPGRLFRSGELSGLDDEGRATLRRLGITDVADLRAAREVARRGPGRVPDGVEVHLLPFPDLGEHEAGTDDQAPHEHAFQRLLTGGAEQSAESVDEAATRYMIDEYRQFPTRNGAQRALHRVISLLGAGRAVLTHCFAGKDRTGFVVATVLEAIGVDRDVIVADFLRSNDAAPALRAQISAMIAQRQDTELTPEVVTWTEARLSDGVLGVREEYLAAARQTIDEKFGSLQAYLRDAGVGEADVQRLRAALLA*

In [77]:
def get_six_frame(dnaSeq):
    """
    get 6 frame translation of qseq_dna
    """
    qtrans_lst=[]
    for strand in [1,-1]:
        for gap in range(3):
            start, end = gap, len(dnaSeq)
            while end%3 != gap:
                end -= 1
            if strand == 1:
                subSeq = dnaSeq[gap:end]
            elif strand == -1 :
                subSeq = dnaSeq[gap:end].reverse_complement()
            qtrans_lst.append(subSeq.translate(table=11))
    return qtrans_lst

In [76]:
fnaFilepath = "/home/mitsuki/altorf/denovo/trg-trace/align/mycobacterium/g062726/g063019-cds4882.fna"
rec_lst = [rec for rec in SeqIO.parse(fnaFilepath, "fasta")]

In [78]:
for seq in get_six_frame(rec_lst[0].seq):
    print(seq)

MTEALRELSGAWNFRDVSDGAPALKPGRLFRSGELSGLDDDGRATLSRLGITDVADLRAAREVARRGPGLVPDGVEVHLLPFPDLGEQDAGTDDAAPHEHAFQRLLTGEGVEESEQSVNEAATRYMIDEYRQFPTRNGAQRALHRVISLLADGHSVLTHCFAGKDRTGFVVATVLEAIGIDRDTILADFLRSNDAAPALRAQISAMIAQRQDAELTPEVVTWTEARLSDGVLGVREVYLAAARQTIDEEFGSLDAYLRAASVSESDVERLREALLA*
*LRRCENCRARGTFVTSPTAHRRSNPGGCSAPAN*AGSTTTAARR*AGWASPTSPICGRPARSRGAGRGWFPTASRCTCCPSPISASRTPGPTTRRRTNTPSSACSPARASRNRSSRSTRPPPAT*STNTANSQRVTGRSGRCTASSRCWPTGIRC*RTASPARTAPASWWRPCWRRSASTATPSWPTSCAATTPRPPCGRRSPR*SRSARTPS*PRRW*PGPRRGCPTACSGSARSTWPPRAKPSTRNSGRWTPTCAPRASASQTWSACARRCSP
D*GVARTVGRVELS*RLRRRTGAQTRAAVPLRRTERARRRRPRDAEPAGHHRRRRSAGGPRGRAARAGAGSRRRRGAPVALPRSRRAGRRDRRRGAARTRLPAPAHRRGRRGIGAVGQRGRHPLHDRRIPPIPNA*RGAAGVAPRHLAAGRRAFGADALLRRQGPHRLRGGDRAGGDRHRPRHHPGRLPAQQRRRARPAGADLRDDRAAPGRRADPGGGDLDRGAAVRRRARGPRGLPGRRAPNHRRGIRVAGRLLARRERQRVRRGAPARGAARL
SGEQRLAQALHV*LADARGAQVGVQRPEFLVDGLARGGQVDLADPEHAVGQPRLGPGHHLRGQLGVLALRDHRGDLRPQGGRGVVAAQEVGQDGVAVDADRLQHGRHHEAGAVLAGEAVRQHRMPVGQQRDDAVQRPLRPVTRWELAVFVDHVAGGGLVDRLLRFLDA

In [79]:
for seq in get_six_frame(rec_lst[1].seq):
    print(seq)

MPEALRELSGAWNFRDVADGAPMLRPGRLFRSGELSGLDDEGRATLRRLGITDVADLRAAREVARRGPGRVPDGVEVHLLPFPDLGEHEAGTDDQAPHEHAFQRLLTGDGGRAVGGVRRRGRHPLHDRRIPAIPNA*RGAASAAPGHLAAGRRACGAHPLLRRQGPHRVRGGDGARSDRRRPRRHRRRLSAQQRRRARAARADLGDDRAAPGHRADPGGGDLDRGAAVRRCAGGARGVPGRRSANHRREVRVAAGLPARRRGRRGRRATPARGAARL
CLRRCENCPARGTFVTSPTVRPCCGRAGCSGPAS*AGSTTRAARRCAGWASPTSPTCARPARWPGAARAGFPTGSRCTCCPFPISASTRPAPTTRRRTSTPSSGCSPATGAEQSAESVDEAATRYMIDEYRQFPTRNGAQRALHRVISLLGAGRAVLTHCFAGKDRTGFVVATVLEAIGVDRDVIVADFLRSNDAAPALRAQISAMIAQRQDTELTPEVVTWTEARLSDGVLGVREEYLAAARQTIDEKFGSLQAYLRDAGVGEADVQRLRAALLA*
A*GAARTVRRVELS*RRRRCAHAAAGPAVPVRRAERARRRGPRDAAPAGHHRRRRPARGPRGGPARPGPGSRRGRGAPAALSRSRRARGRHRRPGAARARLPAAAHRRRGPSSRRSPSTRPPPAT*STNTGNSQRVTGRSERCTGSSRCWAPGVRCSPTASPARTAPGSWWRRCSKRSASTATSSSPTFCAATTPRPRCARRSRR*SRSARTPS*PRRW*PGPRRGCPTVCWGCARSTWPPLGKPSTRSSGRCRPTCATPGSARQTCNACARRCSP
QASSAARRRCTSASPTPASRR*ACSDPNFSSMVCRAAARYSSRTPSTPSDSRASVQVTTSGVSSVSWRCAIIAEICARSAGAASLLRRKSATMTSRSTPIASSTVATTNPVRSLPAKQWVSTARPAPSSEMTRCSARCAPLRVGNCRYSSIM*RVAASSTDSADCSA

In [9]:
def get_rec(target, family, orfid, ext):
    assert ext=="fna" or ext=="faa"
    
    fp = "/data/mitsuki/data/denovo/{0}/annotation/refseq/family/{2}/{1}.{2}".format(target, family, ext)
    for rec in SeqIO.parse(fp, "fasta"):
        if rec.id == orfid:
            return rec
    return None

In [ ]:
family6623
query = "g063032-cds2443"

In [11]:
get_rec("mycobacterium", "family6623", "g063032-cds2443", "faa") #当然1フレーム目の翻訳産物

SeqRecord(seq=Seq('MAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAA...GLP', SingleLetterAlphabet()), id='g063032-cds2443', name='g063032-cds2443', description='g063032-cds2443 WP_080674231.1 hypothetical protein [Mycobacterium kansasii]', dbxrefs=[])

In [3]:
seqFilepath = "../helper/extract.out"
rec_lst = []
for rec in SeqIO.parse(seqFilepath, "fasta"):
    rec_lst.append(rec)

In [4]:
rec_lst

[SeqRecord(seq=Seq('AGCGGGCGACGTTCCGAATGTTCGGTCCGCCGCCCGCTAGCACGGGACTCGGTT...GTG', SingleLetterAlphabet()), id='g062726-NC_002944.2:[438217,', name='g062726-NC_002944.2:[438217,', description='g062726-NC_002944.2:[438217, 438511),0', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAGCTGGCGGAACTACAAAAGCAAATTGTTCGCAGAAATGCGAAGCGGGCGGC...TGA', SingleLetterAlphabet()), id='g063032-cds2443', name='g063032-cds2443', description='g063032-cds2443 lcl|NC_022663.1_cds_WP_080674231.1_2445 [locus_tag=MKAN_RS29640] [db_xref=GeneID:31546045] [protein=hypothetical protein] [protein_id=WP_080674231.1] [location=complement(2811380..2811718)] [gbkey=CDS]', dbxrefs=[])]

In [ ]:
AGDVPNVRSAAR*HGTRLPSVHLAAWVGLGDLTTLPVAAPKGP*TAIRNYAGPQHFCHFRGYDSAWVPSTVLGARIGRSNLLFRDLGLTGLP

In [5]:
for seq in get_six_frame(rec_lst[0].seq):
    print(seq)

SGRRSECSVRRPLARDSVTKRASRGCVGWPRRSHDASGCSPTQRAVDTFRNSQLRRPATLLPFSRL*LRVGAEHRAGRPDREIEPSLPDRPWPHRASV
AGDVPNVRSAAR*HGTRLPSVHLGAAWVGLGDLTTLPVAAPPKGP*TPSAIRNYAGPQHFCHFRGYDSAWVPSTVLGARIGRSNLLFRIDLGLTGLP
RATFRMFGPPPASTGLGYQACISGLRGLASAISRRFRLQPHPKGRRHLPQFAITQARNTSAIFAAMTPRGCRAPCWAPGSGDRTFSSGSTLASPGFR
HGSPVRPRSIRKRRFDLPIRAPSTVLGTHAES*PRKWQKCCGPA*LRIAEGVYGPLGGAATGSVVRSPRPTHAAPRCTLGNRVPC*RAADRTFGTSPA
RKPGEAKVDPEEKVRSPDPGAQHGARHPRGVIAAKMAEVLRACVIANCGRCLRPFGWGCNRKRREIAEANPRSPEMHAW*PSPVLAGGGPNIRNVAR
TEAR*GQGRSGREGSISRSGRPARCSAPTRSHSRENGRSVAGLRNCELRKVSTALWVGLQPEAS*DRRGQPTQPRDARLVTESRASGRRTEHSERRP


In [6]:
for seq in get_six_frame(rec_lst[1].seq):
    print(seq)

IAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAAAPPKGPSLPSALRNYAGPQHFCHYRGYDSAWVPGTVLGARVGRTNLLFRIDLGLTGLP*
*LAELQKQIVRRNAKRAATEGFESPPANTVLGYQACISGLRGLASALLRHFRLQPHPKDRRYHLHFAITQARNTSAIIAAMTPRGCREPCWAPGSGERTFSSGSTLASPGFR
SWRNYKSKLFAEMRSGRRPKDSNRRPPTRFSVTKRAFRGCVGWPRRFCDTSGCSPTQRTVATICTSQLRRPATLLPLSRL*LRVGAGNRAGRPGRENEPSLPDRPWPHRASV
SRKPGEAKVDPEEKVRSPDPGAQHGSRHPRGVIAAIMAEVLRACVIAKCRW*RRSFGWGCSRKCRKSAEANPRSPEMHAW*PRTVLAGGDSNPSVAARFAFLRTICFCSSASY
TEAR*GQGRSGREGSFSRPGRPARFPAPTRSHSRDNGRSVAGLRNCEVQMVATVLWVGLQPEVSQKRRGQPTQPRNARLVTENRVGGRRFESFGRRPLRISANNLLL*FRQL
HGSPVRPRSIRKRRFVLPTRAPSTVPGTHAES*PR*WQKCCGPA*LRSADGSDGPLGGAAAGSVAKAPRPTHAAPKCTLGNREPCWRAAIRILRSPPASHFCEQFAFVVPPA


In [9]:
print(get_six_frame(rec_lst[1].seq)[0])

IAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAAAPPKGPSLPSALRNYAGPQHFCHYRGYDSAWVPGTVLGARVGRTNLLFRIDLGLTGLP*
